In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
(include-book "chat-lib"
              :ttags ((:quicklisp) (:quicklisp.osicat) (:quicklisp.dexador) 
                      (:http-json) (:llm-client) (:mcp-client)))


TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "http-json" ...):  The ttag
note just printed to the terminal indicates a modification to ACL2.
To avoid this warning, supply an explicit :TTAGS argument when including
the book "/workspaces/verified-agent/src/http-json".


TTAG NOTE (for included book): Adding ttag :QUICKLISP.DEXADOR from book /home/acl2/books/quicklisp/dexador.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "http-json" ...):  The ttag
note just printed to the terminal indicates a modification to ACL2.
To avoid this warning, supply an explicit :TTAGS argument when including
the book "/workspaces/verified-agent/src/http-json".


TTAG NOTE (for included book): Adding ttag :HTTP-JSON from book /workspaces/verified-agent/src/http-json.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "http-json" ...):  The ttag
note just printed to the terminal indicates a modification to ACL2.
To avoid this warning, s

In [3]:
(chat "Hello! You are you?  What can you do?")


You: Hello! You are you?  What can you do?

LLM Error: An HTTP request to "http://host.docker.internal:1234/v1/chat/comple\
tions" returned 400 bad request.

#(123 10 32 32 32 32 34 101 114 114 111 114 34 58 32 123 10 32 32 32
32 32 32
  32 32 34 109 101 115 115 97 103 101 34 58 32 34 78 111 32 109 111
100 101 108
  115 32 108 111 97 100 101 100 46 32 80 108 101 97 115 101 32 108
111 97 100
  32 97 32 109 111 100 101 108 32 105 110 32 116 104 101 32 100 101
118 101 108
  111 112 101 114 32 112 97 103 101 32 111 114 32 117 115 101 32 116
104 101 32
  39 108 109 115 32 108 111 97 100 39 32 99 111 109 109 97 110 100
46 34 44 10
  32 32 32 32 32 32 32 32 34 116 121 112 101 34 58 32 34 105 110 118
97 108 105
  100 95 114 101 113 117 101 115 116 95 101 114 114 111 114 34 44 10
32 32 32
  32 32 32 32 32 34 112 97 114 97 109 34 58 32 34 109 111 100 101 108
34 44 10
  32 32 32 32 32 32 32 32 34 99 111 100 101 34 58 32 110 117 108 108
10 32 32
  32 32 125 10 125)


ACL2 !>>(REDEF+)

TTAG NOTE: 

In [4]:
(chat "Show a few theorems about natural number arithmetics")


You: Show a few theorems about natural number arithmetics

LLM Error: An HTTP request to "http://host.docker.internal:1234/v1/chat/comple\
tions" returned 400 bad request.

#(123 10 32 32 32 32 34 101 114 114 111 114 34 58 32 123 10 32 32 32
32 32 32
  32 32 34 109 101 115 115 97 103 101 34 58 32 34 78 111 32 109 111
100 101 108
  115 32 108 111 97 100 101 100 46 32 80 108 101 97 115 101 32 108
111 97 100
  32 97 32 109 111 100 101 108 32 105 110 32 116 104 101 32 100 101
118 101 108
  111 112 101 114 32 112 97 103 101 32 111 114 32 117 115 101 32 116
104 101 32
  39 108 109 115 32 108 111 97 100 39 32 99 111 109 109 97 110 100
46 34 44 10
  32 32 32 32 32 32 32 32 34 116 121 112 101 34 58 32 34 105 110 118
97 108 105
  100 95 114 101 113 117 101 115 116 95 101 114 114 111 114 34 44 10
32 32 32
  32 32 32 32 32 34 112 97 114 97 109 34 58 32 34 109 111 100 101 108
34 44 10
  32 32 32 32 32 32 32 32 34 99 111 100 101 34 58 32 110 117 108 108
10 32 32
  32 32 125 10 125)


ACL2 !>>(REDEF

In [5]:
(chat "What can you do with ACL2?  Try something and show me what you get.")


You: What can you do with ACL2?  Try something and show me what you
get.

LLM Error: An HTTP request to "http://host.docker.internal:1234/v1/chat/comple\
tions" returned 400 bad request.

#(123 10 32 32 32 32 34 101 114 114 111 114 34 58 32 123 10 32 32 32
32 32 32
  32 32 34 109 101 115 115 97 103 101 34 58 32 34 78 111 32 109 111
100 101 108
  115 32 108 111 97 100 101 100 46 32 80 108 101 97 115 101 32 108
111 97 100
  32 97 32 109 111 100 101 108 32 105 110 32 116 104 101 32 100 101
118 101 108
  111 112 101 114 32 112 97 103 101 32 111 114 32 117 115 101 32 116
104 101 32
  39 108 109 115 32 108 111 97 100 39 32 99 111 109 109 97 110 100
46 34 44 10
  32 32 32 32 32 32 32 32 34 116 121 112 101 34 58 32 34 105 110 118
97 108 105
  100 95 114 101 113 117 101 115 116 95 101 114 114 111 114 34 44 10
32 32 32
  32 32 32 32 32 34 112 97 114 97 109 34 58 32 34 109 111 100 101 108
34 44 10
  32 32 32 32 32 32 32 32 34 99 111 100 101 34 58 32 110 117 108 108
10 32 32
  32 32 125 10 125)

